- next数组就是一个前缀表（prefix table）
- 前缀表的作用
  - 前缀表是用来回退的，记录了模式串与主串不匹配时，模式串应该从哪里开始重新匹配
  - 基于不匹配的位置， 要看它的前一个字符的前缀表的数值是多少

ref: https://blog.csdn.net/gmynebula/article/details/125239503 

In [ ]:
# 计算前缀表（获得next数组）

s = "kkskkyds"
# 1 创建前缀表，长度和模式串s相同，初始值为0
prefix = [0]*len(s)
# 2 查找最长公共前后缀（傻找就完了）
# 从最长的前后缀开始找，依次找到只有一个字符
for i in range(1, len(s)):   # i = 1 to n-1
    for j in range(i, 0, -1):
        str1 = s[0:j]   # 前缀
        str2 = s[i-j+1:i+1]  # 后缀
        if str1 == str2:
            prefix[i] = j
            break              # 跳出j循环



# To be contined...

In [4]:
s = "kkskkyds"
i = 3
j = i
str1 = s[0:j]
print(str1)
str2 = s[i-j+1:i+1]
print(str2)

kks
ksk


In [2]:
# 暴力解法

def brute_search(text:str, pattern:str) -> int:
    i, j = 0, 0
    while i < len(text):
        if text[i] == pattern[j]:
            i += 1
            j += 1
            if j == len(pattern):
                return i - j 
        else:
            i = i - j + 1
            j = 0
    return -1


text = 'abcbcglx'
pattern = 'bcgl'

text = 'aabaabaafa'
pattern = 'aabaaf'

res = brute_search(text, pattern)
print(res)


3


In [15]:
def baoli(s:str, p:str) -> int:
    j = 0
    for i in range(0, len(s)):   # 错误示例，原因：使用for循环，每次无论if什么条件，都会+1，如果是几次判断两者相等，i就会自动前进几位，下一次判断如果不相等，i不会回退！所以应该用while
        print(s[i],p[j])
        if s[i] == p[j]:
            j += 1
            if j == len(p)-1:
                return i-j
        else:
            j = 0
        print(i, j)
    return -1


def baoli_correction(s:str, p:str) -> int:
    for i in range(0, len(s)):
        j,k = i,0
        while s[j] == p[k]:   # 这里需要用while，可以一直判断；如果用if，只判断一次
            j += 1
            k += 1
            if k == len(p)-1:
                return i
        print(j, k)
        print(s[j], p[k])
    return -1

        
text = 'abcbcglx'
pattern = 'bcgl'

res = baoli_correction(text, pattern)
print(res)


# 至此，找到了暴力的两种解法：brute_search 和 baoli_correction, 体会while和for循环的区别

0 0
a b
3 2
b g
2 0
c b
3


通过暴力算法的两种实现，重新梳理KMP算法：
暴力算法：模式串每次不匹配需要回退到0，主串每次不匹配需要回退到上一轮开始匹配的下一位，kmp主要是充分利用不匹配字符前匹配子串的信息，做到主串不回退，同时尽可能减少模式串的回退

举两个极端例子

例1:
aaaaaaab
aaaaaab
匹配到最后，a和b不匹配，此时，前面6个a全部匹配，已经匹配的6和a的最大公共子串是5个a，只要移一位，然后主串指针不动，a和模式串第6个a开始比较

例2:
abcdefabcdefac
abcdefac
匹配到最后，b和c不匹配，此时，前面abcdefa全部匹配，最大公共子串是a，模式串移动6位，到第一个a和主串第二个a匹配（充分利用已经匹配的信息），然后，正式的比较是主串的b和模式串的b，然后依次向后

**假设next数组告诉我们：主串指针元素需要和模式串第几个开始下一轮匹配**

j表示模式串不匹配的位置，next[j]表示下一次模式串与主串指针元素匹配的位置
例1中，j = 6, next[j] = 5
例2中，j = 7, next[j] = 1

In [ ]:
def str_str(s: str, p: str, next: list) -> int:
    i, j = 0, 0
    while i < len(s):
        if s[i] == s[j]:
            i += 1
            j += 1
            if j == len(p) - 1:
                return i - j
        else:
            if j == 0:
                i += 1
            else:
                j = next[j]
    return -1
